### Normalizar Informacion VENTAS

In [4]:
import pandas as pd
import numpy as np
import datetime
import re
from datetime import timedelta
from datetime import date

In [5]:
archivo_excel = "C:/Users/contr/OneDrive/Desarrollo Diario/18. Libromatico/LibromaticoALL.xlsx"
nombre_hoja = 'Libromatico'
df = pd.read_excel(archivo_excel, sheet_name=nombre_hoja)

In [6]:
cols = [    "oficina",    "fecha_venta",    "marca",    "modelo",    "año",    "kms",    "costo",    "venta",    "spread",    "utilidad",    "d_stock_f_venta",    "documento",    "nro",    "n_venta",    "n_credito_factura",    "cliente",    "vendedor",    "margen",    "cdo",    "vin",    "patente_usado",    "nro_cot",    "forma_de_pago",    "rut",    "digito",    "tipo_venta",    "categoria",    "marca_vpp",    "modelo_u",    "patente_nuevo",    "fecha_entrega",    "mpp",    "fono",    "fono_2",    "movil",    "email",    "vendedor_2",    "cliente_2",    "chasis",    "categoria_flota",    "email_2",    "fono_casa",    "fono_oficina",    "fono_celular",    "direccion",    "ciudad",    "comuna",    "bono_gm_psa_t_med_nissan_tcl",    "bono_gmac_t_med_distribuidor_maf",    "bono_dealer_finan_nissan",    "bono_compensacion",    "bono_finan_gm_finan_distribuidor",    "bono_flota_camiones_credi_nissan",    "bono_flota_otros",    "bono_dealer_fin",    "bono_chevy_plan_credi_distribuidor",    "bono_oculto",    "unidad_liquidada",    "agendado_en_taller",    "quien_paga_seguro",    "valor_seguro",    "nro_motor",    "color"]
df.columns = cols

In [7]:
df['rut_cliente'] = df['rut'].astype(str) + '-' + df['digito'].astype(str)

In [8]:
df['vin'] = df['vin'].str.strip().str.upper().replace('[^A-Z0-9]+', np.nan, regex=True)
df['vin'] = df['vin'].str.replace(' ', '')
df['vin'] = df['vin'].str.replace('O', '0')

In [9]:
df['oficina'] = df['oficina'].str.upper()
def asignar_unidad_negocios(oficina):
    if "NISSAN" in oficina:
        return "NISSAN"
    elif "TOYOTA" in oficina:
        return "TOYOTA"
    elif "DYP" in oficina:
        return "DYP"
    else:
        return "CHEVROLET"

df["unidad_negocios"] = df["oficina"].apply(asignar_unidad_negocios)

In [10]:
df['patente_usado'] = df['patente_usado'].str[:6].str.strip().str.upper().replace('[^A-Z0-9]+', np.nan, regex=True)
df['patente_nuevo'] = df['patente_nuevo'].str.replace("-", "")

In [11]:
df.loc[:, 'patente_consolidado'] = np.where(df['patente_nuevo'].isnull(), df['patente_usado'], df['patente_nuevo'])

In [12]:
# Seleccionar columnas "col1" y "col2"

df_new = df[['vin', 'patente_consolidado', 'cdo', 'unidad_negocios','oficina','fecha_venta', 'marca', 'modelo', 'año', 'kms', 'costo',
       'venta', 'documento', 'nro','n_venta', 'n_credito_factura', 'cliente',  'rut_cliente','tipo_venta', 'categoria', 'modelo_u', 
       'fecha_entrega', 'mpp', 'cliente_2','movil', 'fono', 'fono_2', 'email',
        'chasis', 'categoria_flota']]


In [13]:
mask = (~df_new['documento'].str.contains("N.Credito")) &  (~df_new['documento'].str.contains("N.Debito"))
df_new = df_new[mask]

In [14]:
df_new.sort_values(["fecha_venta","nro"], ascending=[False,False], inplace=True)
df_ultimoRegistro = df_new.groupby("vin").first().reset_index()


In [15]:
df_ultimoRegistro = df_ultimoRegistro.astype(str)
df_ultimoRegistro["fecha_venta"] = pd.to_datetime(df_ultimoRegistro["fecha_venta"], format="%Y-%m-%d")
df_ultimoRegistro['fecha_entrega'] = pd.to_datetime(df_ultimoRegistro['fecha_entrega'], format='%Y-%m-%d', errors='coerce')
df_ultimoRegistro.to_excel("UltimoRegistroLibrmaticoALL.xlsx", index=False)